In [46]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#import xgboost

df = pd.read_csv("WorldCupMatches.csv")

np.random.seed(0)

In [2]:
wc_teams = ['Australia','Iran','Japan','Korea','Saudi Arabia',
            'Egypt','Morocco','Nigeria','Senegal','Tunisia',
            'Costa Rica','Mexico','Panama','Argentina','Brazil',
            'Colombia','Peru','Uruguay','Belgium','Croatia','Denmark',
            'England','France','Germany','Iceland','Poland','Portugal',
            'Russia','Serbia','Spain','Sweden','Switzerland']
df = (df[df.Year!=2018])
features = (df.drop(['Datetime', 'Home Team Initials' , 'Away Team Initials', 'MatchID','RoundID','Referee','Assistant 1','Assistant 2','Attendance','Stadium','City','Win conditions','Half-time Home Goals','Half-time Away Goals','Win','Draw','Loss',"Year"],axis=1))

In [3]:
isHome = lambda x:pd.Series(df["Home Team Name"] == x)
isAway = lambda x:pd.Series(df["Away Team Name"] == x)
isWin = lambda x:pd.Series(df["Win"] == x)
isStage = lambda x:pd.Series(df["Stage"] == x )

Nation = "Brazil"

print("Home Appearances = " + str(isHome(Nation).sum()))
print("Away Appearances = " + str(isAway(Nation).sum()))
print("Total Apperances = " + str(isHome(Nation).sum() + isAway(Nation).sum()))
#print("Win = " + str(isWin(Nation).sum()))
print("Win Percentage = {0:.1f}%".format(100*isWin(Nation).sum()/(isHome(Nation).sum() + isAway(Nation).sum())) )

#print((df["Win"] == "Germany").count())
features[((features["Away Team Name"] == "Brazil") |(features["Home Team Name"] == "Brazil"))  & (features['Stage'] == "Final" )]

Home Appearances = 82
Away Appearances = 26
Total Apperances = 108
Win Percentage = 69.4%


,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name
135,Final,Brazil,5,2,Sweden
167,Final,Brazil,3,1,Czechoslovakia
231,Final,Brazil,4,1,Italy
515,Final,Brazil,0,0,Italy
579,Final,Brazil,0,3,France
643,Final,Germany,0,2,Brazil


In [4]:
#print(df[(df["Stage"] == "Quarter-finals") & (df["Home Team Name"].str.contains(Nation) | df["Away Team Name"].str.contains(Nation))])
features['Home Win'] = features["Home Team Goals"] > features["Away Team Goals"]
features['Home Team Name'] = features['Home Team Name'].replace('Germany FR', 'Germany')
features['Away Team Name'] = features['Away Team Name'].replace('Germany FR', 'Germany')

wc_home = features[features['Home Team Name'].isin(wc_teams)]
wc_away = features[features['Away Team Name'].isin(wc_teams)]
final = pd.concat((wc_home,wc_away))

final = final.drop(['Home Team Goals','Away Team Goals','Stage'],axis=1)
final.drop_duplicates()
#features[(features['Home Team Name'] == "Italy") & (features['Stage'] == "Final")]
#one_hot = pd.get_dummies(features)
#print(final)
#final["Home Team Name"] 

,Home Team Name,Away Team Name,Home Win
0,France,Mexico,True
4,Argentina,France,True
8,Uruguay,Peru,True
10,Argentina,Mexico,True
11,Brazil,Bolivia,True
13,Uruguay,Romania,True
14,Argentina,Chile,True
15,Argentina,USA,True
16,Uruguay,Yugoslavia,True
17,Uruguay,Argentina,True


In [5]:
encoded = pd.get_dummies(final)
encoded["Home Team Name_Egypt"] = 0
encoded["Away Team Name_Iceland"] = 0
encoded["Home Team Name_Iceland"] = 0
encoded["Away Team Name_Panama"] = 0
encoded["Home Team Name_Panama"] = 0
labels = encoded['Home Win'].values

#encoded["is_train"]=np.random.uniform(0,1,len(encoded)) <= .7

#features.head(10)

features = encoded.loc[:, encoded.columns != "Home Win"]

In [6]:
#train,test = encoded[encoded['is_train'] == True],encoded[encoded['is_train'] == False]

train_features,test_features,train_labels,test_labels = train_test_split(features,labels,test_size = 0.25,random_state = 42)

print('Number of Features in the training data:', train_features.shape)
print('Number of Labels in the training data:', train_labels.shape)
print('Number of Features in the test data:', test_features.shape)
print('Number of Labels in the test data:',test_labels.shape)

Number of Features in the training data: (723, 148)
Number of Labels in the training data: (723,)
Number of Features in the test data: (242, 148)
Number of Labels in the test data: (242,)


In [334]:
#encoded = encoded.drop(['Home Win', 'is_train'])
#labels = list(labels).pop()
#home_win = train['Home Win'].values

In [335]:
#train[labels]

In [8]:
#print(train[labels])

NameError: name 'train' is not defined

In [100]:
clf = RandomForestClassifier(n_jobs=-1, random_state = 3, min_samples_split = 2)
#clf = DecisionTreeClassifier(max_depth = 100)
#clf = LogisticRegression()
clf.fit(train_features,train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=3, verbose=0, warm_start=False)

In [101]:
predictions = clf.predict(test_features)
#errors = abs(predictions - test_labels)
#eval_model = test[["Home Win","Predict"]]
from sklearn.metrics import accuracy_score
#print("Train Score: " + str(accuracy_score(pred,train["Home Win"].values)))
print("Test Score: " + str(accuracy_score(test_labels,predictions)*100))
#accuracy_score(test["Home Win"],test["Predict"])

Test Score: 71.07438016528926


In [62]:
sklearn.metrics.confusion_matrix(test_labels,predictions)

ValueError: Found input variables with inconsistent numbers of samples: [242, 723]

In [36]:
#(np.sum(eval_model["Home Win"] == eval_model["Predict"]))/(eval_model.count())

In [37]:
#print(test[labels])

In [305]:
wc_teams = ['Australia','Iran','Japan','Korea','Saudi Arabia',
            'Egypt','Morocco','Nigeria','Senegal','Tunisia',
            'Costa Rica','Mexico','Panama','Argentina','Brazil',
            'Colombia','Peru','Uruguay','Belgium','Croatia','Denmark',
            'England','France','Germany','Iceland','Poland','Portugal',
            'Russia','Serbia','Spain','Sweden','Switzerland']
wc_home = features[features['Home Team Name'].isin(wc_teams)]
wc_home = wc_home.drop(columns='Stage')
wc_away = features[features['Away Team Name'].isin(wc_teams)]
wc_away = wc_away.drop(columns='Stage')
wc_teams = pd.concat((wc_home,wc_away))
wc_teams.drop_duplicates()
wc_teams.count()
wc_teams
final_pred = pd.get_dummies(wc_teams)
final_pred = final_pred.drop(["Home Team Goals","Away Team Goals","Home Team Win"],axis = 1)

KeyError: 'Home Team Name'

In [102]:
group = {"Home Team Name" : ["Russia","Egypt","Russia","Uruguay","Uruguay","Saudi Arabia","Morocco","Portugal","Portugal","Iran","Spain","Iran","France","Peru","Denmark","France","Australia","Denmark","Argentina","Croatia","Argentina","Nigeria","Nigeria","Iceland","Costa Rica","Brazil","Brazil","Serbia","Serbia","Switzerland","Germany","Sweden","Korea Republic","Germany","Korea Republic","Mexico","Belgium","Tunisia","Belgium","England","Panama","England","Colombia","Poland","Japan","Poland","Japan","Senegal"],
           "Away Team Name" : ["Saudi Arabia", "Uruguay","Egypt","Saudi Arabia","Russia","Egypt","Iran","Spain","Morocco","Spain","Morocco","Portugal","Australia","Denmark","Australia","Peru","Peru","France","Iceland","Nigeria","Croatia","Iceland","Argentina","Croatia","Serbia","Switzerland","Costa Rica","Switzerland","Brazil","Costa Rica","Mexico","Korea Republic","Mexico","Sweden","Germany","Sweden","Panama","England","Tunisia","Panama","Tunisia","Belgium","Japan","Senegal","Senegal","Colombia","Poland","Colombia"]}
WC = pd.DataFrame.from_dict(group)
#WC["Home Team Name"] = "Russia"
#WC["Away Team Name"] = "Saudi Arabia"
predict_set = pd.get_dummies(WC)
WC_col = features.iloc[:0]
WC_col = WC_col.append(predict_set,sort=False)
WC_col.fillna(value = 0,inplace = True)
#predict_set
WC_col[:5]
#set(WC_col.columns) - set(features.columns)
predictions = clf.predict(WC_col)
predictions_proba = clf.predict_proba(WC_col)
for x in range(len(predictions)):
    print(str(predictions[x]) + " " + str(predictions_proba[x]))
#for i in predictions:
#    print(i)

True [0.06666667 0.93333333]
True [0.23333333 0.76666667]
True [0.43333333 0.56666667]
True [0.13333333 0.86666667]
True [0. 1.]
False [0.5 0.5]
False [0.8 0.2]
False [0.73333333 0.26666667]
False [0.8 0.2]
False [0.93333333 0.06666667]
True [0.3 0.7]
True [0.425 0.575]
True [0.35 0.65]
False [0.73333333 0.26666667]
True [0.4 0.6]
True [0.25 0.75]
False [0.5 0.5]
True [0.1 0.9]
True [0.1702381 0.8297619]
False [0.7 0.3]
True [0.1952381 0.8047619]
False [0.8 0.2]
False [0.8 0.2]
False [0.5 0.5]
False [0.8 0.2]
False [0.83333333 0.16666667]
True [0.1 0.9]
True [0.23333333 0.76666667]
False [0.76 0.24]
False [0.55 0.45]
True [0.44989177 0.55010823]
False [0.5 0.5]
False [0.9 0.1]
True [0. 1.]
False [0.5 0.5]
False [0.55 0.45]
True [0.3 0.7]
False [0.75 0.25]
True [0.2 0.8]
True [0.40666667 0.59333333]
True [0.3 0.7]
True [0.265 0.735]
False [0.5 0.5]
True [0.3 0.7]
False [1. 0.]
True [0.1 0.9]
False [0.93333333 0.06666667]
False [0.7 0.3]


In [14]:
#print(WC_col.columns)

In [54]:
eliminations = {"Home Team Name" : ["Russia","Uruguay","France","Peru","Brazil","Serbia","England","Belgium"],
           "Away Team Name" : ["Portugal","Spain","Argentina","Iceland","Germany","Mexico","Colombia","Poland"]}
WC = pd.DataFrame.from_dict(eliminations)
#WC["Home Team Name"] = "Russia"
#WC["Away Team Name"] = "Saudi Arabia"
predict_set = pd.get_dummies(WC)
WC_col = features.iloc[:0]
WC_col = WC_col.append(predict_set,sort=False)
WC_col.fillna(value = 0,inplace = True)
#predict_set
WC_col[:5]
#set(WC_col.columns) - set(features.columns)
predictions = clf.predict(WC_col)
predictions_proba = clf.predict_proba(WC_col)
#for x in range(len(predictions)):
#    print(str(predictions[x]) + " " + str(predictions_proba[x]))
for i in predictions:
    print(i)

True
False
True
True
False
True
True
False


In [55]:
quarter = {"Home Team Name" : ["Russia","Spain","France","Peru"],
           "Away Team Name" : ["Germany","Serbia","England","Poland"]}
WC = pd.DataFrame.from_dict(quarter)
#WC["Home Team Name"] = "Russia"
#WC["Away Team Name"] = "Saudi Arabia"
predict_set = pd.get_dummies(WC)
WC_col = features.iloc[:0]
WC_col = WC_col.append(predict_set,sort=False)
WC_col.fillna(value = 0,inplace = True)
#predict_set
WC_col[:5]
#set(WC_col.columns) - set(features.columns)
predictions = clf.predict(WC_col)
predictions_proba = clf.predict_proba(WC_col)
#for x in range(len(predictions)):
#    print(str(predictions[x]) + " " + str(predictions_proba[x]))
for i in predictions:
    print(i)

False
False
False
False


In [56]:
semi = {"Home Team Name" : ["Germany","Serbia"],
           "Away Team Name" : ["England","Poland"]}
WC = pd.DataFrame.from_dict(semi)
#WC["Home Team Name"] = "Russia"
#WC["Away Team Name"] = "Saudi Arabia"
predict_set = pd.get_dummies(WC)
WC_col = features.iloc[:0]
WC_col = WC_col.append(predict_set,sort=False)
WC_col.fillna(value = 0,inplace = True)
#predict_set
WC_col[:5]
#set(WC_col.columns) - set(features.columns)
predictions = clf.predict(WC_col)
predictions_proba = clf.predict_proba(WC_col)
for x in range(len(predictions)):
    print(str(predictions[x]) + " " + str(predictions_proba[x]))

True [0.48022345 0.51977655]
False [0.63091091 0.36908909]


In [108]:
final = {"Home Team Name" : ["Belgium"],
           "Away Team Name" : ["Brazil"]}
WC = pd.DataFrame.from_dict(final)
#WC["Home Team Name"] = "Russia"
#WC["Away Team Name"] = "Saudi Arabia"
predict_set = pd.get_dummies(WC)
WC_col = features.iloc[:0]
WC_col = WC_col.append(predict_set,sort=False)
WC_col.fillna(value = 0,inplace = True)
#predict_set
WC_col[:5]
#set(WC_col.columns) - set(features.columns)
predictions = clf.predict(WC_col)
predictions_proba = clf.predict_proba(WC_col)
for x in range(len(predictions)):
    print(str(predictions[x]) + " " + str(predictions_proba[x]))

False [0.56 0.44]
